# FlTextureGL Rendering Artifact Reproduction

This notebook reproduces [Flutter Issue #179322](https://github.com/flutter/flutter/issues/179322).

## Prerequisites

1. **Enable GPU Runtime**: Click `Runtime` → `Change runtime type` → Select `T4 GPU`
2. **Install TurboVNC Viewer** on your local machine: https://turbovnc.org/
3. **Create ngrok account** (free): https://ngrok.com/

## Steps

1. Run all cells in order
2. Connect via VNC using the provided address
3. Open terminal in VNC desktop and run the Flutter app

In [ ]:
#@title 1. Check GPU availability
!nvidia-smi

In [ ]:
#@title 2. Install system dependencies
%%bash
apt-get update -qq
apt-get install -y -qq \
    clang \
    cmake \
    ninja-build \
    pkg-config \
    libgtk-3-dev \
    liblzma-dev \
    libepoxy-dev \
    git \
    curl \
    unzip \
    xz-utils \
    libglu1-mesa \
    > /dev/null 2>&1
echo "System dependencies installed."

In [ ]:
#@title 3. Install Flutter SDK
%%bash
if [ ! -d "/content/flutter" ]; then
    git clone https://github.com/flutter/flutter.git -b stable /content/flutter --depth 1
    echo "Flutter cloned."
else
    echo "Flutter already exists."
fi

# Add to PATH
export PATH="/content/flutter/bin:$PATH"
flutter precache --linux
flutter doctor -v

In [ ]:
#@title 4. Clone reproduction repository
%%bash
export PATH="/content/flutter/bin:$PATH"

if [ ! -d "/content/fl_texture_repro" ]; then
    git clone https://github.com/hummer98/fl-texture-repro.git /content/fl_texture_repro
    echo "Repository cloned."
else
    echo "Repository already exists."
fi

cd /content/fl_texture_repro/example
flutter pub get

In [ ]:
#@title 5. Setup VNC + VirtualGL for GPU-accelerated desktop
!pip install git+https://github.com/demotomohiro/remocolab.git -q

In [ ]:
#@title 6. Start VNC Server (requires ngrok authtoken)
#@markdown This will prompt you to enter your ngrok authtoken interactively.
#@markdown Get it from: https://dashboard.ngrok.com/get-started/your-authtoken

import remocolab
remocolab.setupVNC()

## 7. Connect via VNC

After running the cell above, you'll see connection instructions.

1. Open TurboVNC Viewer
2. Connect using the displayed address
3. Enter the password when prompted

## 8. Run Flutter App in VNC Desktop

Once connected to the VNC desktop, open a terminal and run:

```bash
export PATH="/content/flutter/bin:$PATH"
cd /content/fl_texture_repro/example
flutter pub get
vglrun flutter run -d linux
```

The `vglrun` wrapper ensures OpenGL runs on the GPU.

## 9. Observe the Issue

1. Watch the animated texture
2. Look for black shadows/artifacts on buttons
3. Click buttons to trigger redraws
4. Artifacts should appear intermittently

In [ ]:
#@title (Optional) Check OpenGL renderer
%%bash
# This should show NVIDIA GPU when using vglrun
apt-get install -y -qq mesa-utils > /dev/null 2>&1
echo "Without VirtualGL:"
glxinfo | grep "OpenGL renderer" || echo "GLX not available in headless mode"
echo ""
echo "Note: With VirtualGL (vglrun), it will show NVIDIA GPU."

## Troubleshooting

### VNC connection fails
- Ensure ngrok authtoken is correct
- Check if GPU runtime is enabled
- Try restarting the runtime

### Flutter build fails
- Run `flutter doctor` to check for missing dependencies
- Ensure all system packages are installed

### No artifacts visible
- The issue is intermittent and may not appear immediately
- Try clicking buttons rapidly
- The issue is more visible with NVIDIA-specific OpenGL behavior

### Colab T4 GPU vs Jetson
- The artifact may behave differently on T4 vs Jetson
- Both use NVIDIA drivers but may have different OpenGL state handling
- If not reproducible on Colab, refer to the video in the GitHub issue